In [116]:
from __future__ import print_function

import os.path
import pickle

import numpy as np
import pandas as pd
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import seaborn as sns
pd.set_option('display.max_columns', None)

In [28]:
flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
creds = flow.run_local_server(port=0)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=31984095153-kph5j29muu11g829pfi3jnncqknk5qeo.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A39465%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets.readonly&state=sUrVoHcv9WeVA1Kj0iOINwpqO7hmYf&access_type=offline


In [ ]:
service = build('sheets', 'v4',credentials= creds)
sheet = service.spreadsheets()

In [62]:
SAMPLE_SPREADSHEET_ID = '1e1tWLI0vD05bUj-wLWicOnl0iU-GWz0aaWEtRDlTQ2M'

cols = "AF"
rows = 40

SAMPLE_RANGE_NAME = 'A1:'+ cols + str(rows)

In [63]:
result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range=SAMPLE_RANGE_NAME).execute()
values = result.get('values', [])

In [136]:
data = pd.DataFrame(values)
data.columns = data.iloc[0,:]
data = data.iloc[1:,:]
data.head()

,Sygnatura czasowa,Nick,Sex,Age,Adress,Family size,Parent status,Mother education,Father education,Mother job,Father job,Why do you study on MiNI?,Who is your legal guardian?,How long is your comute (in minutes),Study time (in hours),Failed courses,Do you take extra lessons?,Do your family supports your education?,"Do you take courses that you pay for? (dancing, PE, etc...)","Do you have extra activities? (organisations, sport, etc... )",Did you attend preschool?,Do you have internet access at home?,Quality of family relationships,Free time after school,Going out with friends,Workday alcohol consumption,Weekend alcohol consumption,Health,Number of school absences,3rd semester grade,4rd semester grade,Grade up to this date
1,2020-04-18 22:36:19,testuś,Male,19,Rural,Less or equal than 3,living apart,middle school,middle school,civil services,other,school reputation,other,15-30,10 >,1,No,No,Yes,Yes,No,No,2,3,2,4,2,5,13,3.14,4.19,3.90
2,2020-04-18 22:40:00,testuś2,Female,24,Urban,more than 3,living toogether,none,middle school,other,home,other,Mother,60 >,5-10,1,Yes,Yes,No,No,No,Yes,3,3,3,3,3,3,4,4,4,4
3,2020-04-19 10:49:35,test3,Male,21,Urban,more than 3,living toogether,primary education,none,healthcare,teacher,course preference,Father,15-30,2-5,2,No,Yes,Yes,No,No,No,2,3,1,5,5,1,53,4,5,3


In [162]:
def parse(data):
    """
    Parsing data to the same format as in "students-all.csv"
    
    returns: nicks array and dataframe
    """
    nicks = np.array(data['Nick'])

    data = data.iloc[:, 2:]

    column_names = [
        'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob',
        'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures',
        'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'internet',
        'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences',
        'G1', 'G2', 'G3'
    ]

    data.columns = column_names

    # deafults
    data['school'] = np.repeat("GP", data.shape[0])
    data['major'] = np.repeat("mat", data.shape[0])
    data['higher'] = np.repeat("yes", data.shape[0])

    # corect order
    data = data[[
        'school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu',
        'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime',
        'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities',
        'nursery', 'higher', 'internet', 'famrel', 'freetime', 'goout', 'Dalc',
        'Walc', 'health', 'absences', 'G1', 'G2', 'G3', 'major'
    ]]
    
    data['age'] = pd.to_numeric(data['age'])
    data['sex'] = np.where(data.sex == 'Male', 'M', 'F')
    data['address'] = np.where(data.address == 'Rural', 'R', 'U')
    data['famsize'] = np.where(data.famsize == 'more than 3', 'GT3', 'LE3')
    data['Pstatus'] = np.where(data.Pstatus == 'living apart', 'A', 'T')

    d1 = {
        'none': 0,
        'primary education': 1,
        'middle school': 2,
        'high school': 3,
        'higher education': 4
    }

    data['Medu'] = [d1[item] for item in list(data.Medu)]
    data['Fedu'] = [d1[item] for item in list(data.Fedu)]

    d2 = {
        'teacher': 'teacher',
        'healthcare': 'health',
        'civil services': 'civil',
        'home': 'at_home',
        'other': 'other'
    }

    data['Mjob'] = [d2[item] for item in list(data.Mjob)]
    data['Fjob'] = [d2[item] for item in list(data.Fjob)]

    d3 = {
        'close to home': 'home',
        'school reputation': 'reputation',
        'course preference': 'course',
        'other': 'other'
    }

    data['reason'] = [d3[item] for item in list(data.reason)]
    
    d4 = {'Mother':'mother', 'Father':'father', 'other':'other'}
    
    data['guardian'] = [d4[item] for item in list(data.guardian)]

    d5 = {'< 15':1, '15-30':2, '30-60':3,'60 >':4}
    
    data['traveltime'] = [d5[item] for item in list(data.traveltime)]
    
    d6 = {'< 2':1, '2-5':2, '5-10':3,'10 >':4}
    
    data['studytime'] = [d6[item] for item in list(data.studytime)]
    
    data['G1'] = (pd.to_numeric(data.G1)/5)*20 
    data['G2'] = (pd.to_numeric(data.G2)/5)*20 
    data['G3'] = (pd.to_numeric(data.G3)/5)*20 
    
    
    return data

In [163]:
d = parse(data)

/home/jakwisn/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [164]:
d

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,major
1,GP,M,19,R,LE3,A,2,2,civil,other,reputation,other,2,4,1,No,No,Yes,Yes,No,yes,No,2,3,2,4,2,5,13,12.56,16.76,15.6,mat
2,GP,F,24,U,GT3,T,0,2,other,at_home,other,mother,4,3,1,Yes,Yes,No,No,No,yes,Yes,3,3,3,3,3,3,4,16.00,16.00,16.0,mat
3,GP,M,21,U,GT3,T,1,0,health,teacher,course,father,2,2,2,No,Yes,Yes,No,No,yes,No,2,3,1,5,5,1,53,16.00,20.00,12.0,mat
